### Desafio apresentado pela empresa MipWise para os discentes da disciplina de Modelagem e Otimização Aplicada do Programa de Pós-Graduação em Inteligência Artificial Aplicada do Instituto Federal de Goiás (IFG).
* Docente - Prof. Dr. Eduardo Noronha
* Discente - Lucas Elias de andrade Cruvinel (20232011270241)

### Bibliotecas e Funções

In [1]:
import numpy as np
import pandas as pd
import pulp

#### Leitura dos dados
* parameters: parâmetros gerais do problema, como capacidade de estoque e tempo máximo de envelhecimento.
* items_data: informações sobre os itens, como quantidade mínima e máxima de pedido.
* procurement_costs_data: custos de aquisição dos itens por período.
* demand_data: demanda dos itens por período.
* inventory_data: informações sobre estoque, como custos de armazenamento e estoque inicial

In [2]:
# Carrega os dados dos arquivos Excel
parameters = pd.read_excel(io="Dataset_Desafio_v2.xlsx", sheet_name="parameters", index_col='Name')
items_data = pd.read_excel(io="Dataset_Desafio_v2.xlsx", sheet_name="items")
procurement_costs_data = pd.read_excel(io="Dataset_Desafio_v2.xlsx", sheet_name="procurement_costs")
demand_data = pd.read_excel(io="Dataset_Desafio_v2.xlsx", sheet_name="demand")
inventory_data = pd.read_excel(io="Dataset_Desafio_v2.xlsx", sheet_name="inventory")

#### Prob: Cria o problema de minimização de custos usando a biblioteca pulp

In [3]:
# Cria o problema de minimização de custos usando a biblioteca pulp
prob = pulp.LpProblem(name="Minimização de custos", sense=pulp.LpMinimize)

c:\Projetos\MipWise-Dogs\.venv\Lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


#### Define as configurações iniciais:
* BIG_M: Constante alta para utilizar como restrição para o transporte.
* N_PRODUCTS: Número de produtos utilizados.
* N_PERIODS: Número de períodos utilizados.
Infelizmente tive que colocar alguns limites pois se não sempre acontece estouro de memória e não consegue calcular.

#### Define os parâmetros iniciais:
* MAX_AGING_TIME: Tempo máximo de envelhecimento em estoque.
* SUPPLIER_EXPEDITION_CAPACITY: Capacidade de expedição do fornecedor por período.
* WAREHOUSE_RECEIVING_CAPACITY: Capacidade de recebimento do depósito da fábrica.
* SUPPLIER_INVENTORY_CAPACITY: Capacidade de estoque do fornecedor.
* WAREHOUSE_INVENTORY_CAPACITY: Capacidade de estoque local.

In [4]:
BIG_M = 1e4

# Extrai os valores dos parâmetros relevantes do DataFrame parameters
N_PRODUCTS = items_data.shape[0]
N_PERIODS = len(procurement_costs_data['Period ID'].unique())

MAX_AGING_TIME = parameters.loc['Max Aging Time'].iloc[0]
SUPPLIER_EXPEDITION_CAPACITY = parameters.loc['Supplier Expedition Capacity'].iloc[0]
WAREHOUSE_RECEIVING_CAPACITY = parameters.loc['Warehouse Receiving Capacity'].iloc[0]
SUPPLIER_INVENTORY_CAPACITY = parameters.loc['Supplier Inventory Capacity'].iloc[0]
WAREHOUSE_INVENTORY_CAPACITY = parameters.loc['Warehouse Inventory Capacity'].iloc[0]

#### Reorganizar os dados em dicionários:
* $P$: Custo unitário de aquisição (Produtos) para cada item e período.
* $D$: Demanda para cada item e período.
* $M$: Estoque mínimo (M) para cada item e período.

* $CS$: Custo unitário de armazenamento no fornecedor (CS) para cada item.
* $CW$: Custo unitário de armazenamento na fábrica (CW) para cada item.

* $MinP$: Quantidade mínima de items armazenados.
* $MaxP$: Quantidade máxima de items armazenados.
* $MinT$: Minimo de transferências por item.

In [5]:
# Cria dicionários para armazenar informações extraídas de DataFrames

P = {(row["Item ID"], row["Period ID"]): row["Unit Cost"] for _, row in procurement_costs_data.iterrows()}
D = {(row["Item ID"], row["Period ID"]): row["Demand Qty."] for _, row in demand_data.iterrows()}
M = {(row["Item ID"], row["Period ID"]): row["Min Inventory"] for _, row in demand_data.iterrows()}

CS = {row["Item ID"]: row["Unit Holding Cost"] for _, row in inventory_data.iterrows() if row["Site ID"] == "S"}
CW = {row["Item ID"]: row["Unit Holding Cost"] for _, row in inventory_data.iterrows() if row["Site ID"] == "WH"}

MinP = {row["Item ID"]: row["Min Order Qty."] for _, row in items_data.iterrows()}
MaxP = {row["Item ID"]: row["Max Order Qty."] for _, row in items_data.iterrows()}
MinT = {row["Item ID"]: row["Min Transfer Qty."] for _, row in items_data.iterrows()}

#### Define as variáveis de decisão
* $X_{i, t}$: Quantidade de cada item ($i$) comprada em cada período ($t$).
* $W_{i, t}$: Quantidade de cada item ($i$) no estoque da fábrica em cada período ($t$).
* $S_{i, t}$: Quantidade de cada item ($i$) no estoque do fornecedor em cada período ($t$).
* $T_{i, t}$: Quantidade de cada item ($i$) transportada do fornecedor para a fábrica em cada período ($t$).
* $Y_{i, t}$: Variável binária que indica se houve transferência de um item ($i$) do fornecedor para a fábrica em um período ($t$).
* $Z_{i, t}$: Variável binária que indica se houve compra de um item ($i$) em um período ($t$).

In [6]:
X = pulp.LpVariable.dicts(name="X",
    indices=((f'B{i}', t) for i in range(1, N_PRODUCTS+1) for t in range(0, N_PERIODS+1)),
    lowBound=0,
    cat='Integer'
)

W = pulp.LpVariable.dicts(name="W",
    indices=((f'B{i}', t) for i in range(1, N_PRODUCTS+1) for t in range(0, N_PERIODS+1)),
    lowBound=0,
    cat='Integer'
)

S = pulp.LpVariable.dicts(name="S",
    indices=((f'B{i}', t) for i in range(1, N_PRODUCTS+1) for t in range(0, N_PERIODS+1)),
    lowBound=0,
    cat='Integer'
)

T = pulp.LpVariable.dicts(name="T",
    indices=((f'B{i}', t) for i in range(1, N_PRODUCTS+1) for t in range(0, N_PERIODS+1)),
    lowBound=0,
    cat='Integer'
)

Y = pulp.LpVariable.dicts(name="Y",
    indices=((f'B{i}', t) for i in range(1, N_PRODUCTS+1) for t in range(0, N_PERIODS+1)),
    lowBound=0,
    upBound=1,
    cat='Integer'
)

Z = pulp.LpVariable.dicts(name="Z",
    indices=((f'B{i}', t)for i in range(1, N_PRODUCTS+1) for t in range(0, N_PERIODS+1)),
    lowBound=0,
    upBound=1,
    cat='Integer'
)

#### Adiciona os itens iniciais do inventário de $W$ e $S$.

In [7]:
initial_inventory_wh = {row["Item ID"]: row["Opening Inventory"] for _, row in inventory_data.iterrows() if row["Site ID"] == "WH"}
for i in range(1, N_PRODUCTS+1):
    W[(f'B{i}', 0)].setInitialValue(initial_inventory_wh.get(f'B{i}', 0))
    W[(f'B{i}', 0)].fixValue()

initial_inventory_s = { row["Item ID"]: row["Opening Inventory"] for _, row in inventory_data.iterrows() if row["Site ID"] == "S"}
for i in range(1, N_PRODUCTS+1):
    S[(f'B{i}', 0)].setInitialValue(initial_inventory_s.get(f'B{i}', 0))
    S[(f'B{i}', 0)].fixValue()
    

#### Função objetivo

Minimizar o custo total gerado durante o periodo informado.

A função é a soma de:
* Custo de aquisição de produtos: $P * X$.
* Custo de armazenamento no supplier: $S * CS$.
* Custo de armazenamento no warehouse: $W * CW$.

In [8]:
prob += pulp.lpSum(
    [P[f'B{i}', t] * X[(f'B{i}', t)] + CS[f'B{i}'] * S[(f'B{i}', t)] + CW[f'B{i}'] * W[(f'B{i}', t)]
    for t in range(1, N_PERIODS+1)
    for i in range(1, N_PRODUCTS+1)
])

#### Restrições de acordo com as regras operacionais referente aos períodos:
Reespectivamente de acordo com o código abaixo:
* Restrições de Aquisição: 
    * Limita o número máximo de diferentes tipos de itens $(Z)$ que podem ser adquiridos em um período $(t)$.
* Restrições de Capacidade de Estoque do Fornecedor: 
    * Assegura que o estoque do fornecedor $(B)$ não exceda sua capacidade em cada período $(t)$.
* Restrições de Capacidade de Estoque da Fábrica: 
    * Garante que o estoque da fábrica $(W)$ não ultrapasse sua capacidade em cada período $(t)$.
* Restrições de Capacidade de Transporte do Fornecedor: 
    * Limita a quantidade total de itens que o fornecedor pode enviar à fábrica $(T)$ em cada período $(t)$.
* Restrições de Capacidade de Recebimento da Fábrica: 
    * Limita a quantidade de diferentes tipos de itens que a fábrica pode receber do fornecedor $(Y)$ em cada período $(t)$.

In [9]:
for t in range(1, N_PERIODS+1):
    prob += pulp.lpSum(Z[(f'B{i}', t)] for i in range(1, N_PRODUCTS+1)) <= WAREHOUSE_RECEIVING_CAPACITY

    prob += pulp.lpSum(S[(f'B{i}', t)] for i in range(1, N_PRODUCTS+1)) <= SUPPLIER_INVENTORY_CAPACITY

    prob += pulp.lpSum(W[(f'B{i}', t)] for i in range(1, N_PRODUCTS+1)) <= WAREHOUSE_INVENTORY_CAPACITY

    prob += pulp.lpSum(T[(f'B{i}', t)] for i in range(1, N_PRODUCTS+1)) <= SUPPLIER_EXPEDITION_CAPACITY

    prob += pulp.lpSum(Y[(f'B{i}', t)] for i in range(1, N_PRODUCTS+1)) <= WAREHOUSE_RECEIVING_CAPACITY

#### Restrições de acordo com as regras operacionais referente aos produtos:

Reespectivamente de acordo com o código abaixo:

* As três primeiras restrições são Restrições de Quantidade de Aquisição $(X)$ de acordo com a variável $Z$. A variável $Z$ indica se houve compra e limita a compra à quantidade máxima (MaxP) quando Z é 1 (houve compra).:
    * Limite Superior: certifica que a quantidade comprada de um item não exceda a quantidade máxima de pedido.
    * Limite Inferior de $(Z)$: garante que, se houver compra de um item (Z=1), a quantidade mínima de pedido seja respeitada.
    * Limite Inferior: certifica que, se não houver compra de um item (Z=0), a quantidade adquirida seja zero.
* Restrições de Balanço de Estoque do Fornecedor $(S)$: 
    * Garante que o estoque do fornecedor em um período seja igual ao estoque do período anterior, acrescido da quantidade comprada e subtraído da quantidade transferida para a fábrica.
* As próximas duas retrições são sobre o Balanço de Estoque da Fábrica $(W)$:
    * Assegura que o estoque da fábrica em um período seja igual ao estoque do período anterior, acrescido da quantidade recebida do fornecedor e subtraído da demanda.
    * Restrição de estoque mínimo da Fábrica: garante que o estoque da fábrica de um item em cada período seja igual ou superior ao estoque mínimo definido para o item.
* As últimas três são referentes a Quantidade de transferência $(T)$:
    * Limite Inferior a partir de $(Y)$: garante que a transferência do fornecedor para a fábrica ocorra apenas se o $Y$ no produto no período $(t)$ $(Y[(f'B{i}', t)])$ for igual a 1. A quantidade transferida será maior ou igual à quantidade mínima de transferência, ou zero caso contrário.
    * Limite Superior: limita a quantidade transferida de um item, em cada período, à capacidade de expedição do fornecedor.
    * Limite Inferior: garante que a transferência, se ocorrer (Y=1), seja de pelo menos uma unidade do item.

In [10]:
for i in range(1, N_PRODUCTS+1):
    prob += X[(f'B{i}', t)] <= MaxP[f'B{i}'] * Z[(f'B{i}', t)]
    prob += X[(f'B{i}', t)] >= 1 * Z[(f'B{i}', t)]
    prob += X[(f'B{i}', t)] >= MinP[f'B{i}'] - BIG_M * (1-Z[(f'B{i}', t)])


    prob += S[(f'B{i}', t-1)] + X[(f'B{i}', t)] == T[(f'B{i}', t)] + S[(f'B{i}', t)]

    prob += W[(f'B{i}', t-1)] + T[(f'B{i}', t)] == D.get((f'B{i}', t), 0) + W[(f'B{i}', t)]
    prob += W[(f'B{i}', t)] >= M.get((f'B{i}', t), 0)

    prob += T[(f'B{i}', t)] >= MinT[f'B{i}'] - BIG_M * (1-Y[(f'B{i}', t)])
    prob += T[(f'B{i}', t)] <= SUPPLIER_EXPEDITION_CAPACITY * Y[(f'B{i}', t)]
    prob += T[(f'B{i}', t)] >= 1 * Y[(f'B{i}', t)]


### Display dos resultados

In [11]:
solverList = pulp.listSolvers(onlyAvailable=True)
solverList

['PULP_CBC_CMD']

#### Utiliza a função Solve para aplicar o Solver

In [12]:
# Resolvendo o problema de otimização utilizando o solver CBC
prob.solve(solver=pulp.getSolver('PULP_CBC_CMD'))

# Imprimindo o status da solução do problema
print("Status:", pulp.LpStatus[prob.status])

# Verificando se a solução é viável
if pulp.LpStatus[prob.status] == 'Infeasible':
    print('\nSem Solução')
else:
    # Imprimindo o valor ótimo da função objetivo
    print("Custo ótimo:", pulp.value(prob.objective))


Status: Optimal
Custo ótimo: 32420.019999999997


In [13]:
# Preparando para extrair os resultados da solução
NewMinP = {}
for t in range(0, N_PERIODS + 1):
    for i in range(1, N_PRODUCTS + 1):
        # Armazenando o estoque mínimo por produto e período
        NewMinP[(f'B{i}', t)] = MinP[f'B{i}']

# Criando listas para armazenar os valores das variáveis de decisão
data = [X, Z, S, T, Y, W]

# Criando um DataFrame a partir dos dados das variáveis de decisão
df = pd.DataFrame.from_records(data, index=['Aquisição', 'Comprar', 'Armazém Fornecedor', 'Transporte', 'Transportar', 'Armazém Fábrica'])
df = df.T # Transpondo o DataFrame

# Função para extrair o valor da variável
def getvalue(x):
    return x.varValue

# Aplicando a função getvalue para obter os valores das variáveis
for key in df.columns:
    df[key] = df[key].apply(getvalue)

# Configurando o índice do DataFrame com Produto e Período
df.index = pd.MultiIndex.from_tuples(df.index, names=['Produto', 'Período'])
df = df.fillna(0) # Preenchendo valores ausentes com 0

# Configurando a exibição do DataFrame
pd.options.display.max_rows = 999
# Exibindo apenas os períodos de 0 a 9
print(df[df.index.get_level_values('Período').isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])])

                 Aquisição  Comprar  Armazém Fornecedor  Transporte  \
Produto Período                                                       
B1      0              0.0      0.0               149.0         0.0   
        1              0.0      0.0                 0.0         0.0   
        2              0.0      0.0                 0.0         0.0   
        3              0.0      0.0                 0.0         0.0   
        4              0.0      0.0                 0.0         0.0   
        5              0.0      0.0                 0.0         0.0   
        6              0.0      0.0                 0.0         0.0   
        7              0.0      0.0                 0.0         0.0   
        8              0.0      0.0                 0.0         0.0   
        9              0.0      0.0                 0.0         0.0   
B2      0              0.0      0.0               196.0         0.0   
        1              0.0      0.0                 0.0         0.0   
      